<a href="https://colab.research.google.com/github/JManuelRG/propiedades_termodinamicas/blob/main/Sustancias%20Puras/Ec_de_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ecuación de Antoine
# Contantes Obtenidas de la base de datos web de NIST:

https://webbook.nist.gov/chemistry/

# Web scrapper:
https://github.com/oscarcontrerasnavas/NIST-web-book-scraping/blob/master/README.md

https://pypi.org/project/beautifulsoup4/


# Instrucciones:

Ejecutar la siguiente función para obtener los parámetros de la ecuación de Antoine.

La celda es un Web Scrapper que buscará el compuesto en la base de datos de NIST y obtendrá los datos para ese compuesto.

In [1]:
# Import get function from requests module because is the function in charge of
# getting the HTTP GET request with the given url.
from requests import get

# Import BeautifulSoup from bs4 because make the html parse and help us to
# handle de DOM.
from bs4 import BeautifulSoup

# Import closing for ensure that any network resource will free when they go out
# of scope.
from contextlib import closing

def get_antoine_coef(Name, Temperature):

    """ Return a list with the coefficients A, B and C if they exist for the
        given Temperature. If not, return None and print it.

    :param Name:
        A string with the name of the compound in English.

    :param Temperature:
        A float number with the temperature in Kelvin.

    :rtype: List

    :return coef with [A, B, C]

    """

    # Obtaining the table using the get_html function showed below. Table is a
    # BeautifulSoup Object.
    table = get_html_table(Name)

    # Extract the rows from the table. Knowing what tags have an HTML table.
    # Also, knowing that the fist row with he table header does not have the
    # class attribute 'exp' so we obtain just the rows with data.
    # The find_all function from BeautifulSoup return a list
    rows = table.find_all('tr', class_='exp')

    # Declaring the lists for storage Temperatures, and coefficients.
    Temperatures, As, Bs, Cs = [], [], [], []

    # Looping over rows to extract and fill As, Bs, and Cs variables because now
    # we are sure the Temperatues is between some range.
    for row in rows:

        # As the rows, we extract the columns for the current row. Knowing that
        # the cols have the <td> tag in HTML as well
        # The find_all function from BeautifulSoup return a list
        cols =  row.find_all('td')

        # First transform the strings into float numbers and put them in their
        # respective list
        As.append(float(cols[1].text))
        Bs.append(float(cols[2].text))
        Cs.append(float(cols[3].text))

        # For the temperatures, we have a range and we need to extract each
        # limit (lower and higher) and put them in an extra list. So
        # Temperatures variable will be a list of lists.
        lower_lim = float(cols[0].text.replace(" ","").split('-')[0])
        higher_lim = float(cols[0].text.replace(" ","").split('-')[1])
        Temperatures.append([lower_lim, higher_lim])


    # Checking if the Temperature gave fits in some interval
    index = None
    for i, interval in enumerate(Temperatures):
        if (interval[0] <= Temperature
            and Temperature <= interval[1]):
            index = i
            break
        else:
            index = None

    if index == None:
        print('Sorry, the data for the given temperature %.2f K does not exist in the Data Base' % Temperature)
        return None
    else:
        A = As[index]
        B = Bs[index]
        C = Cs[index]
        return [A, B, C]


def get_html_table(Name):

    """ Return the html already parsed using the a helper function listed below.

    :param Name:
        A string with the name of the compound in English.

    :rtype: BeautifulSoup Object

    """

    # The name parameter is part of the url. For example, if you want the
    # methane data, the url is
    # https://webbook.nist.gov/cgi/cbook.cgi?Name=methane&Mask=4.
    url = str.format('https://webbook.nist.gov/cgi/cbook.cgi?Name={0}&Mask=4', Name.lower())

    # Function to get the request made, see below.
    raw_html = get_response(url)

    # Parse the html using BeautifulSoup.
    html = BeautifulSoup(raw_html, 'html.parser')

    # Extract the table that contains the data, the table has a specific
    # attributes 'aria-label' as 'Antoine Equation Parameters'.
    table = html.find('table', attrs={'aria-label': 'Antoine Equation Parameters'})

    return table


def get_response(url):

    """ Return the raw_html for parsing later or None if can't reach the page

    :param url:
        The string for the GET request.

    :rtype: BeautifulSoup Object

    :rtype: None if can't reach the website

    """

    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except:
        print('Not found')
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

In [2]:
import pandas as pd
# Obtener las variables de la ecuación de Antoine
print("Constantes de la Ec. de Antoine:")
nombre = input('Escribe el nombre del compuesto en inglés: ')
T = float(input('Escribe la temperatura promedio del compuesto, K: '))
A,B,C = get_antoine_coef(nombre,
                  T)
print(f'Tabla de constantes de Antoine para -{nombre}-, válidos a una temperatura de {T} Kelvin')
print('Obtenidos de la base de Datos NIST:')
print(f'URL = https://webbook.nist.gov/cgi/cbook.cgi?Name={nombre}&Mask=4')
datosAntoine = pd.DataFrame({'Compuesto':nombre,
                             'A':[A],
                             'B':[B],
                             'C':[C]})
datosAntoine.set_index('Compuesto')

Constantes de la Ec. de Antoine:
Escribe el nombre del compuesto en inglés: water
Escribe la temperatura promedio del compuesto, K: 400
Tabla de constantes de Antoine para -water-, válidos a una temperatura de 400.0 Kelvin
Obtenidos de la base de Datos NIST:
URL = https://webbook.nist.gov/cgi/cbook.cgi?Name=water&Mask=4


,A,B,C
Compuesto,,,
water,3.55959,643.748,-198.043


# Valores Puntuales:

# Cálculo de la Presión de saturación de la sustancia:
# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

$$P^{sat} = 10^{A −\frac{B}{T^{sat}+C}}$$

In [3]:
print('Determinación de la Presión de saturación de un sistema:')
Tsat = float(input('Escribe el valor de la temperatura de saturación en °C: '))
Psat = 10**(datosAntoine['A']-datosAntoine['B']/(Tsat + 273.15 + datosAntoine['C']))
print(f"La presión de saturación de '{datosAntoine['Compuesto'][0]}' calculada, es de {Psat[0]} bar")
print(f"Psat = {Psat[0]*100} kPa")

Determinación de la Presión de saturación de un sistema:
Escribe el valor de la temperatura de saturación en °C: 100
La presión de saturación de water calculada, es de 0.7643245377276982 bar
Psat = 76.43245377276982 kPa


# Cálculo de la Temperatura de Saturación de la sustancia:

$$T(°C) = \frac{B}{(A-log_{10}P(bar))}-C$$

In [12]:
import math
print('Determinación de la temperatura de saturación de un sistema:')
Psatc = float(input('Escribe el valor de la presión de saturación en bar: '))

Tsatc = datosAntoine['B']/(datosAntoine['A']-math.log10(Psatc))-datosAntoine['C']

print(f"La temperatura de saturación de '{datosAntoine['Compuesto'][0]}', calculada es de {Tsatc[0]} K")
print(f"Tsat = {Tsatc[0]-273.15} °C")


Determinación de la temperatura de saturación de un sistema:
Escribe el valor de la presión de saturación en bar: 7
La temperatura de saturación de 'water', calculada es de 435.1952957111213 K
Tsat = 162.0452957111213 °C
